In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load all data
#We can use the read_csv() function to load the data and combine the first two columns into a single datetime column to use it as an index.
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])
# summarize
print(dataset.shape)
print(dataset.head())

In [ ]:
from numpy import isnan

# mark all missing values
dataset.replace('?', np.nan, inplace=True)

# make dataset numeric
dataset = dataset.astype('float32')

# fill missing values with a value at the same time one day ago
def fill_missing(values):
    one_day = 60 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if isnan(values[row, col]):
                values[row, col] = values[row - one_day, col]
                
fill_missing(dataset.values)

#Alternatively, we can just remove these nan values:
#remove columns with nan
#dataset = dataset.dropna()

In [ ]:
dataset.shape

In [ ]:
# add a column for for the remainder of sub metering
values = dataset.values
dataset['Sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])
dataset[0:1000]

In [ ]:
# resample data to daily
day_groups = dataset.resample('D')
dataset = day_groups.sum()
dataset[0:1000]

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

ax = plt.gca()

dataset['Global_active_power'].plot(figsize=(13, 5), title = 'Global_active_power', ax = ax)
#dataset['Sub_metering_1'].plot(figsize=(,158), title = 'Global_active_power', ax = ax) 
#dataset['Sub_metering_2'].plot(figsize=(15,8), title = 'Global_active_power', ax = ax) 
#dataset['Sub_metering_3'].plot(figsize=(15,8), title = 'Global_active_power', ax = ax) 
#dataset['Sub_metering_4'].plot(figsize=(15,8), title = 'Global_active_power', ax = ax) 

plt.show()

In [ ]:
# line plot for each variable
plt.figure(figsize=(13,13))
for i in range(len(dataset.columns)):
    plt.subplot(len(dataset.columns), 1, i+1)
    name = dataset.columns[i]
    plt.plot(dataset[name])
    plt.title(name, y=0)
plt.show()

In [ ]:
import statsmodels
from statsmodels.tsa.stattools import adfuller

adf_test = adfuller(dataset['Global_active_power'])

adf_test

# ADF -3.840, p-value = 0.0024
#
#We can see that the ADF value (the first value in the result) is -3.840) and the p-value (the 2nd value) is 0.0024. 
#ADF of less than the value of 0.0024 suggests that we can reject the null hypothesis with a significance
#level of less than 1% (i.e. a low probability that the result is a statistical fluke). 
#Rejecting the null hypothesis means that the process has no unit root, and that the time series is 
#stationary or does not have time-dependent structure.

In [ ]:
from statsmodels.tsa.stattools import kpss

kpss_test = kpss(dataset['Global_active_power'])

kpss_test

#Since the p-value is 0.1, the null hypothesis is not rejected at the usual 5% level.

In [ ]:
from numpy import split
from numpy import array

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

#We will use the first three years of data for training predictive models and the final year for evaluating models.
#Then, the data is divided into weeks that begin on a Monday and end on a Sunday.

# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train, test = data['Global_active_power'][2:-327], data['Global_active_power'][-327:-5] # we take position 2 as this is the first Monday in the dataset
    # restructure into windows of weekly data
    train = array(split(train, len(train)/7))
    test = array(split(test, len(test)/7))
    return train, test
 
# splt the dataset
train, test = split_dataset(dataset)
# check train data
print(train.shape)
# check test
print(test.shape)

In [ ]:
train

In [ ]:
test

In [ ]:
# acf and pacf plots of total power
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

#We can calculate the correlation for time series observations with the use of obervations of lags. 
#We can then create a single figure that contains both an ACF and a PACF plot. 
#The number of lag time steps can be specified, in our case we fix it to 365 days of observations (365 days).
#The ACF plot indicates that there is a strong autocorrelation component,
#The PACF plot indicates that this component is distinct for the approximatelly 1 lag of observations.

# plots
pyplot.figure(figsize=(11,7))
lags = 365 
# acf
axis = pyplot.subplot(2, 1, 1)
plot_acf(dataset['Global_active_power'], ax=axis, lags=lags)
# pacf
axis = pyplot.subplot(2, 1, 2)
plot_pacf(dataset['Global_active_power'], ax=axis, lags=lags)
# show plot
pyplot.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

#Since the plots above are quite dense, we can and change the number of lag observations 
#from 365 to 30 to zoom in the plot.
#We can see that a good starting point would be an autoregressive model with 1 lag obervations used as a parameter.

lags = 30

plot_acf(dataset['Global_active_power'], lags=lags)
plt.show()

In [ ]:
#cross validation

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt

tscv = TimeSeriesSplit(n_splits = 3)
rmse = []
predictions = list()

for train_index, test_index in tscv.split(train):
    cv_train, cv_test = train[train_index], train[test_index]
        
    for t in range(len(train)-1):
        model = ARIMA(train[t], order=(1,0,0)).fit(disp=False)
        yhat = model.predict(len(train[t]), len(train[t])+6)
        predictions.append(yhat)
        rmse.append(mean_squared_error(train[t+1], yhat))
        
print("rmse score: {}".format(np.mean(rmse)))

In [ ]:
from statsmodels.tsa.arima_model import ARMAResults

#Running this prints a summary of the fit model. 
#This summarizes the coefficient values used as BIC and AIC values.

print(ARMAResults.summary(model_fit))

In [ ]:
#this cell is not used anymore

#from statsmodels.tsa.arima_model import ARIMA
#from sklearn.metrics import mean_squared_error

#predictions = list()
#rmse = list()

# arima forecast
#for t in range(len(train)-1):
    # define the model
    #model = ARIMA(train[t], order=(1,0,0))
    # fit the model
    #model_fit = model.fit(disp=False)
    # make forecast
    #yhat = model_fit.predict(len(train[t]), len(train[t])+6)
    #predictions.append(yhat)
    #rmse.append(mean_squared_error(train[t+1], yhat))